## Fit gaussians to the bouts distribution to define directionality

In [1]:
%matplotlib widget
from pathlib import Path

import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, DATASET_LOCATION, LotrExperiment, dataset_folders
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from tqdm import tqdm

COLS = pltltr.COLS

In [11]:
DATASET_LOCATION

PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/My Drive/data/all_source_data/full_ring')

In [8]:
dataset_folders = sorted(
    [f.parent for f in DATASET_LOCATION.glob("*[0-9]_f[0-9]*/*/selected.h5")]
)


In [15]:
newpath = Path("/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring")

In [16]:
list(newpath.glob("*/*/selected.h5"))

[PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/210314_f1/210314_f1_natmov/selected.h5'),
 PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/211207_f5/211207_f5beyes_natmov/selected.h5'),
 PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/210728_f1/210728_f1eyes_clol/selected.h5'),
 PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/211118_f0/211118_f0_spont/selected.h5'),
 PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/210926_f2/210926_f2_gainmod/selected.h5'),
 PosixPath('/Users/vigji/Library/CloudStorage/GoogleDrive-luigi.petrucco@gmail.com/.My Drive/data/all_source_data/full_ring/210314_f0/210314_f0_natmov/selected.h5'

## Load all bout data and compute histograms:

In [2]:
n_bins = 50  # number of bins in the -pi to +pi interval

# Load all bout dataframes:
hist_base_bouts = np.linspace(-np.pi, np.pi, n_bins)
bout_bias_hists = []
for path in dataset_folders:
    exp = LotrExperiment(path)
    h, _ = np.histogram(exp.bouts_df["bias"], hist_base_bouts, density=True)
    bout_bias_hists.append(h)

bout_bias_hists = np.array(bout_bias_hists)
mean_hist = bout_bias_hists.mean(0)

hist_x = (hist_base_bouts[1:] + hist_base_bouts[:-1]) / 2

/var/folders/wp/q156lfy924zg6bqc48v9j2800000gn/T/ipykernel_36938/3067050905.py:12: RuntimeWarning: Mean of empty slice.
  mean_hist = bout_bias_hists.mean(0)
/Users/vigji/miniforge3/envs/rplab/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Fit trimodal distribution to the histogram.

Constrains:
 - Lateral distributions should have the same (absolute) mean and standard deviation
 - Center distribution should have mean 0
 
Initial guesses:
 - Central curve: `mean=0`, `std=0.1` (conservatively low), `amplitude=max(hist)`
 - Lateral curves: `mean=pi` (conservatively high), `std=pi` (conservatively high), `amplitude=max(hist)/50` (conservatively high)

In [3]:
def _gaus(x, a, mn, sigma):
    return a * np.exp(-((x - mn) ** 2) / (2 * sigma ** 2))


def tri_gaus(x, fwd_a, turn_a, turn_mn, fwd_sigma, turn_sigma):
    return (
        _gaus(x, fwd_a, 0, fwd_sigma)
        + _gaus(x, turn_a, turn_mn, turn_sigma)
        + _gaus(x, turn_a, -turn_mn, turn_sigma)
    )


# Amp (center), amp (lateral), mean (center), mean (lateral), std (center), std (lateral)
initial_guesses = [np.max(mean_hist), np.max(mean_hist) / 50, np.pi, 0.1, np.pi]

popt, pcov = curve_fit(tri_gaus, hist_x, mean_hist, p0=initial_guesses)

# Define turning bias as the intercept between the center and the lateral distributions:
min_search = np.arange(-np.pi, np.pi, 0.0001)
turn_bias = min_search[2:][
    (np.diff(tri_gaus(min_search, *popt))[1:] > 0)
    & (np.diff(tri_gaus(min_search, *popt))[:-1] < 0)
][1]
print(np.round(turn_bias, 3))

ValueError: array must not contain infs or NaNs

In [ ]:
f, ax = plt.subplots(figsize=(3, 2), gridspec_kw=dict(bottom=0.2, left=0.2))
for i, h in enumerate(bout_bias_hists):
    ax.bar(
        hist_x,
        h,
        width=hist_x[1] - hist_x[0],
        fc=COLS["ph_plot"],
        lw=0.0,
        alpha=0.08,
        label="_nolabel_",
    )
ax.bar(
    hist_x,
    h,
    width=hist_x[1] - hist_x[0],
    fc=COLS["qualitative"][0],
    lw=0.0,
    alpha=0.2,
    label="single fish",
)
ax.step(
    hist_x,
    np.nanmean(bout_bias_hists, 0),
    lw=1.0,
    c=pltltr.shift_lum(COLS["qualitative"][0], -0.1),
    where="mid",
    label="mean",
)
ax.set(
    # yscale="log",
    **pltltr.get_pi_labels(0.5),
    xlabel="Laterality index",
    ylabel="Prob. density (log(p))",
    ylim=(0, 1.5)
)
plt.plot(hist_x, tri_gaus(hist_x, *popt), lw=1, c=COLS["qualitative"][3], label="fit")

ax.legend(loc=2, bbox_to_anchor=(0.7, 1.1), fontsize=7)
pltltr.despine(ax)
pltltr.savefig("all_bouts_ps_nothresholds")

for l in [-turn_bias, turn_bias]:
    plt.axvline(l, lw=0.5, c=".1")
plt.show()
pltltr.savefig("all_bouts_ps")